In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%autosave 0
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

import sys
sys.path.insert(0,'..')

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')


In [ ]:
# LOAD DATA
#{seqname:[shapearray, sequence, structure]}

import rna_io
data = rna_io.get_all_data('../data/RNA16.react', '../data/RNA16.dbn')
data.pop("R009")
data.pop("23sRNA")
'''
data.pop("ADDRSW")
data.pop("GLYCFN")
data.pop("RNAsep")
data.pop("ZHCV")
data.pop("adenin")
'''
print "done"


In [ ]:
# TRAIN ON N-1, predict on rest
import simushape as sr
import rna_tools
import rna_tools.rnaplfold
import rna_tools.rnafold
import collections
from scipy.stats import pearsonr as pear
import sukosd
import rna_io
import numpy as np
import rna_accuracy
import notebook_helper as nh


data = rna_io.get_all_data('../data/RNA16.react', '../data/RNA16.dbn') # for Sukosd
dataShaker = data 
#dataShaker.pop("R009",None) 
#dataShaker.pop("23sRNA",None)
keysShaker= dataShaker.keys()

dataSukosd = rna_io.get_all_data('../data/RNA16.react', '../data/RNA16.dbn')
keysSukosd = dataSukosd.keys()
print "keysSukosd ", keysSukosd
SukosdShapeList = []

#predictions = list(sr.crosspredict(data, keys, seq_to_db_function=rna_tools.rnashapes, cutoff=0.0000000001))


def correlationOfProbabilities(cross_predictions_list,fig_title, data, Sukosd=None):
    
    #data.pop("R009",None) # we use this with the same trainig set to call run_all_analysis(something, 'Realstructure-Shaker-Z-sametrain')
    #data.pop("23sRNA",None) # we use this with the same trainig set to call run_all_analysis(something, 'Realstructure-Shaker-Z-sametrain')
    pmfereallist = []                                                                                                                                                      
    pmfesimulist = []
    psimureallist = []
    keys= data.keys()
    print "keys : ", keys
    
    # FOR ALL seqyences
    for i, key in enumerate(keys):
        # GET ACCessibility for sequence
        mfelist = rna_tools.rnaplfold.rnaplfold(data[key][1])# Probability of each nucleotide in sequence (data[key][1]) 
        # GET accessibility with ... sequence and real SHAPE data
        realShapelist = rna_tools.rnaplfold.rnaplfold(data[key][1], data[key][0]) 
        #GET ACCESSIBILITY sequcence and simulated shape data
        if Sukosd == None:
            simuShapelist = rna_tools.rnaplfold.rnaplfold(data[key][1], cross_predictions_list[i]) #name of reactvity file 
            
        # compute pearson correlation between probabilities computed between MFE, simu, realshape
        else: simuShapelist = rna_tools.rnaplfold.rnaplfold(data[key][1], cross_predictions_list[i])
        pmfereal = pear(mfelist,realShapelist)[0]
        pmfesimu = pear(mfelist,simuShapelist)[0]
        psimureal = pear(simuShapelist,realShapelist)[0]
        #list of all MFE (simu or real) of all sequences
        pmfereallist.append(pmfereal)
        pmfesimulist.append(pmfesimu)
        psimureallist.append(psimureal)
    correlationBoxplot(pmfereallist, pmfesimulist, psimureallist, keys, fig_title)
    xlabels = ['RealShape', 'SimuShape']
    label_title = 'Comparison of SHAPE computed by'
    nh.reactivitiesBoxplot(cross_predictions_list, data, keys, fig_title, xlabels, label_title)  
    
    
def correlationBoxplot(pmfereallist, pmfesimulist, psimureallist, keys, fig_title):     
    print 
    print ('*******  ', fig_title, '******* ')
    print "Results of correlations : ENS&realShape ENS&simu simu&realShape"
    print
    #print correlations with RNA names
    for i, key in enumerate(keys):
        print "%s\t\t\t%0.2f\t%0.2f\t%0.2f"%(key, pmfereallist[i], pmfesimulist[i], psimureallist[i])
    corrData = [pmfereallist, pmfesimulist, psimureallist]
    axis = ['ENS&RealShape', 'ENS&SimuShape', 'SimuShape&RealShape']
    filesave = "correlation"
    plt_title = 'Correlations between 2 of 3 ENS, SimuShape, RealShape'
    nh.boxplotDraw(corrData, axis, plt_title, fig_title, filesave)
    

#correlationOfProbabilities(list(sr.crosspredict(dataShaker, keysShaker, seq_to_db_function = lambda seq:[rna_tools.rnafold.fold(seq)])), 'RNAfold', dataShaker) # accuracy can be computed for RNAfold that computes a dot plot
#correlationOfProbabilities(list(sr.crosspredict(dataShaker, keysShaker, seq_to_db_function=rna_tools.rnasubopt.rnasubopt)), 'RNAsubopt', dataShaker) # worked for RNAsubopt
#correlationOfProbabilities(list(sr.crosspredictInter(data, keys, seq_to_db_function= lambda seq:[rna_tools.rnafold.fold(seq)])), 'RNAfold')#intermediate

#Sukosd with RNAfold
for i, key in enumerate(keysSukosd):
    structure = rna_tools.rnafold.fold(dataSukosd[key][1])
    SukosdShape = sukosd.sukosd(structure)
    SukosdShapeList.append(SukosdShape)
correlationOfProbabilities(SukosdShapeList, 'RNAfold',dataSukosd) 


#we need both shape and str to compute accuracy
#when we call RNAfold with shape data?
def accuracyCrosspredict(data, keys, Sukosd=None):
    accuracyList = []
    if Sukosd==None:
        reactivityList = list(sr.crosspredict(data, keys, seq_to_db_function = lambda seq:[rna_tools.rnafold.fold(seq)])) 
    for i, key in enumerate(keys): 
        if Sukosd==None:
            react = reactivityList[i]
        else: 
            react = SukosdShapeList[i]
        seq_to_db = rna_tools.rnafold.fold(data[key][1], react=react, return_dotplot=True)
        accuracy1 = rna_accuracy.get_structure_accuracy(['/home/montaser/ShaKer/notebooks/dot.ps'],[data[key][2]])
        accuracyList.append(accuracy1)
    return accuracyList 

def get_accuracy(Sukosd=None):
    if Sukosd != None:
        accuracyShaker = accuracyCrosspredict(dataShaker, keysShaker)
        return accuracyShaker
    else: 
        SukosdShapeList = []
        accuracySukosd = accuracyCrosspredict(dataSukosd, keysSukosd, 1)
        del accuracySukosd[7]
        del accuracySukosd[12]
        return accuracySukosd

acc  = get_accuracy(1) #compute accuracy 
print "acc ", acc

def boxplotAccuracy(accuracyShaker, accuracySukosd, fig_title):
    accuracyData = [accuracyShaker, accuracySukosd]
    axis = ['accuracyShaker', 'accuracySukosd']
    plt_title = "Comparison of accuracy"
    filesave = "Accuracy-Shaker" + fig_title+ "-SukosdMFE"
    nh.boxplotDraw(accuracyData, axis, plt_title, fig_title, filesave)


print "RNA sequence & accuracyShaker & accuracySukosd"    
for k, l1, l2 in zip(keysShaker,accuracyShaker, accuracySukosd):
    print k, l1, l2
boxplotAccuracy(accuracyShaker, accuracySukosd, 'RNAsubopt')
#nh.boxplotAccuracy(accuracyShaker, accuracySukosd, 'RNAfold')
    

# need function sequence -> real structure
#  {seqname:[react, sequence, dotbracketstring]}

trick= { sequence : [stru] for react, sequence, stru in dataSukosd.values() }
get_real_struct = lambda seq : trick[seq] 
#correlationOfProbabilities(list(sr.crosspredict(dataShaker, keysShaker, seq_to_db_function = get_real_struct)), 'Realstructure-Shaker-Z', dataShaker) 


print "keysSukosd ", keysSukosd
#something = {a:b for a,b in sr.modelpredict(dataSukosd, keysSukosd)}
#run_all_analysis(something, 'Realstructure-Shaker-Z-sametrain', 1) #change the model for training


'''
#Sukosd with real strutcure
SukosdShapeList = {}
for i, key in enumerate(keysSukosd):
    structure = dataSukosd[key][2]
    #print "structure : ", structure
    SukosdShape = sukosd.sukosd(structure)
    #print "SukosdShape ", SukosdShape
    SukosdShapeList[key]= SukosdShape
    #print SukosdShapeList

run_all_analysis(SukosdShapeList, 'RealStructure-Sukosd-Z', dataSukosd) 
'''
'''
This is not logical.
averaged mean is necessary 
for i, key in enumerate(keys):
    structures = rna_tools.rnasubopt(data[key][1])
    #print "structures : ", structures
    for j, structure in enumerate(structures):
        print "i : ", i
        print "j : ", j
        print "structure : ", structure
        SukosdShape = sukosd.sukosd(structure)
        print SukosdShape
run_all_analysis(SukosdShape, 'RNAsubopt') 
'''




In [ ]:
asd = lambda: (1,2,3)

a,s,d  = asd()

print a, s

In [ ]:
print os.path.isfile(dp_ps)
print trick

In [ ]:

run_all_analysis(list(sr.crosspredict(data, keys, seq_to_db_function=rna_tools.rnasubopt)), 'RNAsubopt')



In [ ]:
asdkja
asdkkjsa
asda


In [ ]:
import rna_tools as rt

# EVALUATE 
for key,prediction in zip(keys,predictionsSoheila):
    proba = rt.probability(data[key][2],data[key][1])
    annproba = rt.probability(data[key][2],data[key][1], prediction)
    realReactProb = rt.probability(data[key][2],data[key][1],data[key][0])
    print "%s\t%0.9f\t%0.9f\t%0.9f\t%d\t%d\t%d" % (key, proba, annproba,  realReactProb, proba < annproba or realReactProb < annproba, data[key][2] == rt.fold(data[key][1]), realReactProb < annproba)
    

In [ ]:
# todo: subopt, regressor